In [1]:
import mhcflurry
import numpy
import logging
import seaborn
import time
from matplotlib import pyplot

% matplotlib inline
logging.basicConfig(level="DEBUG")

Using TensorFlow backend.
/home/qiuxing/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Predict Using Existing Model

In [25]:
### Try an unsupported allele ###
%time mhcflurry.predict(alleles=["HLA-A0201"], peptides=["SIINFEKL","SIINFEQL"])

CPU times: user 0 ns, sys: 196 ms, total: 196 ms
Wall time: 2.8 s


,Allele,Peptide,Prediction
0,HLA-A0201,SIINFEKL,10672.347656
1,HLA-A0201,SIINFEQL,7828.974121


In [23]:
### a unit to catch unsupported allele error ###
import sys
try:
    print(mhcflurry.predict(alleles=["HLA-A0201"], peptides=["SIINFEKL","SIINFEQL"]))
except:
    print("Unexpected error:", sys.exc_info()[0])

      Allele   Peptide    Prediction
0  HLA-A0201  SIINFEKL  10672.347656
1  HLA-A0201  SIINFEQL   7828.974121


# Load Customized Training Set and Model

In [2]:
kim2014_train = mhcflurry.dataset.Dataset.from_csv(mhcflurry.downloads.get_path("data_kim2014", "bdata.2009.mhci.public.1.txt"))

In [9]:
kim2014_train.alleles

array(['ELA-A1', 'ELA-A1', 'ELA-A1', ..., 'Patr-B2401', 'Patr-B2401',
       'Patr-B2401'], dtype=object)

In [10]:
dfrmli_A0201_test = mhcflurry.dataset.Dataset.from_csv("testset/HLA-A0201")
len(dfrmli_A0201_test)

1251

In [13]:
dfrmli_A0201_test.affinities

array(['-', '-', '-', ..., '+', '+', '+'], dtype=object)

In [3]:
A0201_trainingset = kim2014_train.get_allele("HLA-A0201")
A0201_trainingset

Dataset(n=9565, alleles=['HLA-A0201'])

In [16]:
prediction_model = mhcflurry.class1_allele_specific.Class1BindingPredictor()
prediction_model.hyperparameters

{'activation': 'tanh',
 'batch_normalization': True,
 'batch_size': 128,
 'dropout_probability': 0.0,
 'embedding_output_dim': 32,
 'fraction_negative': 0.0,
 'init': 'glorot_uniform',
 'kmer_size': 9,
 'layer_sizes': [64],
 'loss': 'mse',
 'max_ic50': 50000.0,
 'n_training_epochs': 250,
 'optimizer': 'rmsprop',
 'output_activation': 'sigmoid',
 'pretrain_decay': 'numpy.exp(-epoch)'}

In [18]:
predictions = prediction_model.predict(dfrmli_A0201_test.peptides)
predictions

array([  84.33081818,  197.01031494,  172.89077759, ...,   93.08477783,
        135.23834229,  429.84768677], dtype=float32)

# Evalute Prediction Accuracy

In [20]:
mhcflurry.class1_allele_specific.scoring.make_scores(dfrmli_A0201_test.affinities, predictions)

/home/qiuxing/.local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/home/qiuxing/.local/lib/python2.7/site-packages/scipy/stats/stats.py:250: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


{'auc': nan, 'f1': 0.0, 'tau': 0.073076446246030796}

In [21]:
kim2014_test = mhcflurry.dataset.Dataset.from_csv(
    mhcflurry.downloads.get_path("data_kim2014", "bdata.2013.mhci.public.blind.1.txt"))
test_set = kim2014_test.get_allele("HLA-A0201")
len(test_set)

2126

In [22]:
test_set.affinities

array([  6.94444444e+04,   6.61261200e+00,   7.22181500e+00, ...,
         7.81250000e+04,   1.12300000e+03,   1.30000000e+02])

In [23]:
predictions_new = prediction_model.predict(test_set.peptides)
predictions_new

array([ 422.26077271,   91.14395142,  119.12548065, ...,  226.68408203,
        131.43814087,  140.93244934], dtype=float32)

In [25]:
mhcflurry.class1_allele_specific.scoring.make_scores(test_set.affinities, predictions_new)

{'auc': 0.57255260326485824,
 'f1': 0.70411233701103315,
 'tau': 0.087641280225993323}

# Custom Cross-Validation Methods and Model Parameters

In [27]:
help(mhcflurry.class1_allele_specific.cross_validation.cross_validation_folds)

Help on function cross_validation_folds in module mhcflurry.class1_allele_specific.cross_validation:

cross_validation_folds(train_data, alleles=None, n_folds=3, drop_similar_peptides=False, imputer=None, impute_kwargs={'min_observations_per_allele': 2, 'min_observations_per_peptide': 2}, parallel_backend=None)
    Split a Dataset into n_folds cross validation folds for each allele,
    optionally performing imputation.
    
    Parameters
    -----------
    train_data : mhcflurry.Dataset
    
    alleles : string list, optional
        Alleles to run cross validation on. Default: all alleles in
        train_data.
    
    n_folds : int, optional
        Number of cross validation folds for each allele.
    
    drop_similar_peptides : boolean, optional
        For each fold, remove peptides from the test data that are similar
        to peptides in the train data. Similarity is defined as in the
        similar_peptides function.
    
    imputer : fancyimpute.Solver, optional
     

In [4]:
fold1 = mhcflurry.class1_allele_specific.cross_validation.cross_validation_folds(A0201_trainingset,n_folds=3)
fold2 = mhcflurry.class1_allele_specific.cross_validation.cross_validation_folds(A0201_trainingset,n_folds=5)

In [24]:
fold1

[AlleleSpecificTrainTestFold(allele='HLA-A0201', train=Dataset(n=6376, alleles=['HLA-A0201']), imputed_train=None, test=Dataset(n=3189, alleles=['HLA-A0201'])),
 AlleleSpecificTrainTestFold(allele='HLA-A0201', train=Dataset(n=6377, alleles=['HLA-A0201']), imputed_train=None, test=Dataset(n=3188, alleles=['HLA-A0201'])),
 AlleleSpecificTrainTestFold(allele='HLA-A0201', train=Dataset(n=6377, alleles=['HLA-A0201']), imputed_train=None, test=Dataset(n=3188, alleles=['HLA-A0201']))]

In [6]:
models = mhcflurry.class1_allele_specific.train.HYPERPARAMETER_DEFAULTS.models_grid(
    fraction_negative=[.1],
    layer_sizes=[12],
    impute=[False,True])
models[0]
models[1]

{'activation': 'tanh',
 'batch_normalization': True,
 'batch_size': 128,
 'dropout_probability': 0.0,
 'embedding_output_dim': 32,
 'fraction_negative': 0.1,
 'impute': True,
 'init': 'glorot_uniform',
 'kmer_size': 9,
 'layer_sizes': 12,
 'loss': 'mse',
 'max_ic50': 50000.0,
 'n_training_epochs': 250,
 'optimizer': 'rmsprop',
 'output_activation': 'sigmoid',
 'pretrain_decay': 'numpy.exp(-epoch)'}

In [7]:
df = mhcflurry.class1_allele_specific.train.train_across_models_and_folds(fold1, models, return_predictors=True)

TypeError: 'int' object is not iterable

# Imputation Test

In [ ]:
from fancyimpute import MICE
kim2014_train = mhcflurry.dataset.Dataset.from_csv(mhcflurry.downloads.get_path("data_kim2014", "bdata.2009.mhci.public.1.txt"))
impute_set = kim2014_train.impute_missing_values(MICE(n_imputations=25))
A0203_impute = impute_set.get_allele("HLA-A0203")
A0203_trainingset = kim2014_train.get_allele("HLA-A0203")
print("Before Impute: %d; After Impute: %d" % (len(A0203_trainingset),len(A0203_impute)))

[MICE] Completing matrix with shape (31539, 106)
[MICE] Starting imputation round 1/35, elapsed time 3.706
[MICE] Starting imputation round 2/35, elapsed time 22.248
[MICE] Starting imputation round 3/35, elapsed time 34.405
[MICE] Starting imputation round 4/35, elapsed time 46.069
[MICE] Starting imputation round 5/35, elapsed time 57.038
[MICE] Starting imputation round 6/35, elapsed time 66.790
[MICE] Starting imputation round 7/35, elapsed time 78.032
[MICE] Starting imputation round 8/35, elapsed time 88.780
[MICE] Starting imputation round 9/35, elapsed time 99.075
[MICE] Starting imputation round 10/35, elapsed time 109.121
[MICE] Starting imputation round 11/35, elapsed time 120.484
[MICE] Starting imputation round 12/35, elapsed time 133.577
[MICE] Starting imputation round 13/35, elapsed time 147.796
[MICE] Starting imputation round 14/35, elapsed time 159.606
[MICE] Starting imputation round 15/35, elapsed time 172.604
[MICE] Starting imputation round 16/35, elapsed time 18